# Section 2: RNNs in PyTorch

## Goals
1. Build a simple RNN classifier
2. Learn about PyTorch's in-built RNN modules (LSTM etc.)

(Roughly follows http://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)

In [1]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torchtext
from torchtext.vocab import Vectors, GloVe

### Part 1: Building an RNN sentiment classifier
#### Part 1.1: Generating the data

First we'll generate some toy data. The task will be to recall an integer at a certain position in a sequence. 
For a sequence a<sub>1</sub> a<sub>12</sub> a<sub>3</sub> a<sub>4</sub> a<sub>5</sub> the output might be a<sub>3</sub>.

In [2]:
# number of training examples
n_train = 2000

# number of validation examples
n_val = 1000

# length of each sequence
n_length = 10

# examples per batch
n_batch = 32

# size of the vocabulary
n_vocab = 20

# position to be recalled
answer_pos = n_length-1

# generate random sequences
train_seq = Variable(torch.Tensor(n_train, n_length).random_(0, n_vocab).long())
val_seq = Variable(torch.Tensor(n_val, n_length).random_(0, n_vocab).long())

# choose the correct labels
train_labels = train_seq.clone()[:, answer_pos]
val_labels = val_seq.clone()[:, answer_pos]

# group data into batches
train_iter = []
for i in range(0, n_train, n_batch):
    batch_seq = train_seq[i:i+n_batch]
    batch_labels = train_labels[i:i+n_batch]
    if (batch_seq.size()[0] == n_batch):
        train_iter.append([batch_seq, batch_labels])
    
val_iter = []
for i in range(0, n_val, n_batch):
    batch_seq = val_seq[i:i+n_batch]
    batch_labels = val_labels[i:i+n_batch]
    if (batch_seq.size()[0] == n_batch):
        val_iter.append([batch_seq, batch_labels])

#### Part 1.2 Build the model (version 1)

The RNN module will be a PyTorch model like any other, with init a forward functions. This network:
1. Takes as input the word at a particular point in the sequence, as well as the hidden state at the previous state of the network
2. Uses nn.Embedding to get a vector for the word
3. Concatenate the embedding and the hidden state
4. Apply a linear layer to get the next hidden state
5. Apply a linear layer to get the output
6. Output both 

In [3]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, vocab_size):
        super(RNN, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, input_size)
        self.hidden_size = hidden_size
        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(input_size + hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        embedded = self.embedding(input)
        combined = torch.cat((embedded, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

#### Part 1.3: Train the model

Now we can initialize and train the network:

In [4]:
def train_batch(model, criterion, optim, batch, label):
    # initialize hidden vector
    hidden = Variable(torch.zeros(n_batch, n_hidden))

    # clear gradients
    rnn.zero_grad()

    # calculate forward pass
    for i in range(batch.size()[1]):
        output, hidden = model(batch[:, i], hidden)

    # calculate loss    
    loss = criterion(output, label)

    # backpropagate and step
    loss.backward()
    optim.step()
    
    return loss.data[0]

In [5]:
# training loop
def train(model, criterion, optim):
    for e in range(n_epochs):
        batches = 0
        epoch_loss = 0
        avg_loss = 0
        for batch, label in train_iter:
            batch_loss = train_batch(model, criterion, optim, batch, label)
            batches += 1
            epoch_loss += batch_loss
            avg_loss = ((avg_loss * (batches - 1)) + batch_loss) / batches
        
        print("Epoch ", e, " Loss: ", epoch_loss)
        


In [6]:
# size of the hidden vector
n_hidden = 3

# initialize the network
rnn = RNN(n_vocab, n_hidden, n_vocab, n_vocab)

n_epochs = 30
learning_rate = .05
criterion = nn.NLLLoss()
optim = torch.optim.SGD(rnn.parameters(), lr = learning_rate)

train(rnn, criterion, optim)

('Epoch ', 0, ' Loss: ', 130.83899009227753)
('Epoch ', 1, ' Loss: ', 50.6698834002018)
('Epoch ', 2, ' Loss: ', 25.83748033642769)
('Epoch ', 3, ' Loss: ', 16.157479643821716)
('Epoch ', 4, ' Loss: ', 11.279053911566734)
('Epoch ', 5, ' Loss: ', 8.458589285612106)
('Epoch ', 6, ' Loss: ', 6.6770948469638824)
('Epoch ', 7, ' Loss: ', 5.472653165459633)
('Epoch ', 8, ' Loss: ', 4.613367971032858)
('Epoch ', 9, ' Loss: ', 3.9737212359905243)
('Epoch ', 10, ' Loss: ', 3.4811954759061337)
('Epoch ', 11, ' Loss: ', 3.091461520642042)
('Epoch ', 12, ' Loss: ', 2.7760953195393085)
('Epoch ', 13, ' Loss: ', 2.5161177664995193)
('Epoch ', 14, ' Loss: ', 2.2984154634177685)
('Epoch ', 15, ' Loss: ', 2.1136592347174883)
('Epoch ', 16, ' Loss: ', 1.9550437536090612)
('Epoch ', 17, ' Loss: ', 1.817494686692953)
('Epoch ', 18, ' Loss: ', 1.6971564795821905)
('Epoch ', 19, ' Loss: ', 1.5910500511527061)
('Epoch ', 20, ' Loss: ', 1.4968388937413692)
('Epoch ', 21, ' Loss: ', 1.412664981558919)
('Epoch

#### Part 1.4: Test the model

Testing the model is similar to training it:

In [8]:
def test_batch(batch, label):
    if (batch.size()[0] != n_batch):
        return 0, 0
    
    # initialize hidden state
    hidden = Variable(torch.zeros(n_batch, n_hidden))
    
    # calculate forward pass
    for i in range(batch[0].size()[0]):
        output, hidden = rnn(batch[:, i], hidden)
        
    # calculate predictions
    _, pred = output.max(1)

    # calculate number of correct predictions
    correct = (pred == label).long().sum().data[0]
    return correct, n_batch

Then calculate the total score by looping through the batches:

In [9]:
# Test loop

batch_num = 0
correct = 0
total = 0
for i in range(len(val_iter)):
    batch, label = val_iter[i]
    batch_correct, batch_size = test_batch(batch, label)
    batch_num += 1
    correct += batch_correct
    total += batch_size
    
print("Percent correct: ", correct / total)

('Percent correct: ', 1)


### Part 2: Using PyTorch RNN modules

PyTorch's RNN capabilities live [here](http://pytorch.org/docs/master/nn.html#recurrent-layers). We can use it as follows (note that the input is batched along the **second** dimension):

In [10]:
n_input = 10
n_hidden = 20
n_layers = 2
n_batch = 3
n_length = 5
rnn = nn.LSTM(n_input, n_hidden, n_layers)
input = Variable(torch.randn(n_length, n_batch, n_input))
h0 = Variable(torch.randn(n_layers, n_batch, n_hidden))
c0 = Variable(torch.randn(n_layers, n_batch, n_hidden))
output, hn = rnn(input, (h0, c0))
print(output, hn)

(Variable containing:
(0 ,.,.) = 

Columns 0 to 8 
   0.2262 -0.0881  0.3527 -0.0746 -0.4060  0.2240  0.0088  0.0468 -0.1581
  0.3601 -0.0734  0.1484 -0.1260 -0.2714  0.0355  0.3520  0.2879  0.2156
 -0.2936 -0.0323 -0.0249 -0.1940 -0.2356  0.2073  0.1414  0.0973 -0.4495

Columns 9 to 17 
  -0.3059  0.2751  0.0088 -0.0933 -0.1202 -0.4326  0.0313 -0.1018  0.0146
 -0.1165 -0.3390 -0.0310 -0.0821 -0.0676  0.3112 -0.4281  0.3291 -0.0294
  0.2613 -0.5185 -0.4656  0.0107 -0.0335  0.3838 -0.1666  0.4846 -0.4016

Columns 18 to 19 
  -0.0253 -0.0641
  0.2978 -0.2378
  0.2789 -0.1280

(1 ,.,.) = 

Columns 0 to 8 
   0.1780 -0.0809  0.1363 -0.0823 -0.2173  0.0859  0.0469  0.0392 -0.1529
  0.1640 -0.1072  0.2449 -0.1035 -0.2663  0.1396  0.0673  0.1351  0.0464
 -0.0974 -0.0674  0.0780 -0.1079 -0.0513  0.3051  0.0788  0.0656 -0.1932

Columns 9 to 17 
  -0.1391  0.2459  0.0167  0.0307 -0.1048 -0.1051 -0.0197 -0.1417 -0.0278
  0.0538 -0.0898  0.0028  0.0194 -0.0913  0.2075 -0.1135  0.2126  0.0112
  0.2

We'll define a custom module to apply this module to our problem. This module will embed each integer, then apply the LSTM to the sequence, and then apply a linear and a softmax to get probabilities for each class:

In [15]:
class MyLSTM(nn.Module):
    def __init__(self, embedding_size, hidden_size, output_size, vocab_size, n_layers):
        super(MyLSTM, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, n_layers)
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        # embed the input integers
        embedded = self.embedding(input)
        print(embedded.size()) # batch x length x embedding
        
        # put the batch along the second dimension
        embedded = embedded.transpose(0, 1)
        
        # apply the LSTM
        output, hidden = self.lstm(embedded, hidden)
        
        # apply the linear and the softmax
        output = self.softmax(self.linear(output))

        return output, hidden

Training and testing are essentially the same as before, except that we no longer need to manually loop in the forward pass:

In [16]:
def train_batch(model, criterion, optim, batch, label):
    # initialize hidden vectors
    hidden = (Variable(torch.zeros(n_layers, n_batch, n_hidden)), Variable(torch.zeros(n_layers, n_batch, n_hidden)))

    # clear gradients
    rnn.zero_grad()

    # calculate forward pass
    output, hidden = model(batch, hidden)

    # calculate loss    
    loss = criterion(output[-1], label)

    # backpropagate and step
    loss.backward()
    optim.step()
    
    return loss.data[0]

In [17]:
# training loop
def train(model, criterion, optim):
    for e in range(n_epochs):
        batches = 0
        epoch_loss = 0
        avg_loss = 0
        for batch, label in train_iter:
            batch_loss = train_batch(model, criterion, optim, batch, label)
            batches += 1
            epoch_loss += batch_loss
            avg_loss = ((avg_loss * (batches - 1)) + batch_loss) / batches
        
        print("Epoch ", e, " Loss: ", epoch_loss)
        


In [18]:
# size of the embeddings and vectors
n_embedding = 128
n_hidden = 128

# number of layers
n_layers = 1

# initialize LSTM
rnn = MyLSTM(n_embedding, n_hidden, n_vocab, n_vocab, n_layers)

n_epochs = 30
learning_rate = .1
criterion = nn.NLLLoss()
optim = torch.optim.SGD(rnn.parameters(), lr = learning_rate)

train(rnn, criterion, optim)

torch.Size([32, 10, 128])


RuntimeError: The size of tensor a (32) must match the size of tensor b (3) at non-singleton dimension 0

In [ ]:
# Test loop

batch_num = 0
correct = 0
total = 0
for i in range(len(val_iter)):
    batch, label = val_iter[i]
    batch_correct, batch_size = test_batch(batch, label)
    batch_num += 1
    correct += batch_correct
    total += batch_size
    
print("Percent correct: ", correct / total)